## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [4]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256]))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10]))
        self.b2 = tf.Variable(tf.zeros([10]))


    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 35.73011 ; accuracy 0.42455
epoch 1 : loss 35.27651 ; accuracy 0.43003333
epoch 2 : loss 34.837433 ; accuracy 0.43456668
epoch 3 : loss 34.411724 ; accuracy 0.43921667
epoch 4 : loss 33.998608 ; accuracy 0.44355
epoch 5 : loss 33.59749 ; accuracy 0.44778332
epoch 6 : loss 33.207893 ; accuracy 0.4521
epoch 7 : loss 32.829403 ; accuracy 0.45618334
epoch 8 : loss 32.461624 ; accuracy 0.46035
epoch 9 : loss 32.104187 ; accuracy 0.46406665
epoch 10 : loss 31.75646 ; accuracy 0.46755
epoch 11 : loss 31.417734 ; accuracy 0.47125
epoch 12 : loss 31.087524 ; accuracy 0.47431666
epoch 13 : loss 30.765596 ; accuracy 0.47801667
epoch 14 : loss 30.451784 ; accuracy 0.4816
epoch 15 : loss 30.145859 ; accuracy 0.48468333
epoch 16 : loss 29.84758 ; accuracy 0.48801666
epoch 17 : loss 29.556946 ; accuracy 0.49168333
epoch 18 : loss 29.273712 ; accuracy 0.49495
epoch 19 : loss 28.99757 ; accuracy 0.49836665
epoch 20 : loss 28.728308 ; accuracy 0.50151664
epoch 21 : loss 28.465647 ; accura